In [1]:
import pickle
import faiss
import torch
from transformers import BertModel, BertTokenizer

# 피클 파일 경로
combined_file_path = "/data/matmang/peS2o_validation/combined_peS2o_validation.pkl"

# 데이터 로더 함수 정의
def load_data(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

# 특정 조건에 맞는 데이터 필터링 함수 정의
def filter_data(data, source_value):
    return [record for record in data if record.get('source') == source_value]

# BERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# 텍스트를 임베딩하는 함수 정의
def embed_texts(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# 데이터 로드 및 필터링
data = load_data(combined_file_path)
filtered_data = filter_data(data, 's2orc/valid')
print(f"Filtered records: {len(filtered_data)}")

# 텍스트 임베딩 생성
texts = [record['text'] for record in filtered_data]
embeddings = embed_texts(texts)

# FAISS 인덱스 생성 및 임베딩 추가
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance
index.add(embeddings)
print(f"Indexed {index.ntotal} records")

# 쿼리 텍스트 임베딩 및 검색
query = "Your query text here"
query_embedding = embed_texts([query])
D, I = index.search(query_embedding, k=5)  # 상위 5개 결과 검색

# 검색 결과 출력
print("Search results:")
for i in range(len(I[0])):
    print(f"Rank {i+1}:")
    print(filtered_data[I[0][i]])
    print(f"Distance: {D[0][i]}")

/home/matmang/anaconda3/envs/faiss-tutorial/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/matmang/anaconda3/envs/faiss-tutorial/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Filtered records: 51323


KeyboardInterrupt: 